In [1]:
import numpy as np
import pandas as pd



from data_prep import data_prep

df = data_prep()

df.info()


/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


<class 'pandas.core.frame.DataFrame'>
Int64Index: 41002 entries, 0 to 41475
Columns: 1885 entries, VehYear to VehBCost_?
dtypes: bool(2), float64(11), int64(1), uint8(1871)
memory usage: 77.3 MB


In [21]:
from sklearn.model_selection import train_test_split

# target/input split
y = df['IsBadBuy']
X = df.drop(['IsBadBuy'],axis = 1)

# setting random state
rs = 10

# Undersampling 
badBuy_indices = df[df['IsBadBuy'] == 0].index

sample_size = sum(df.IsBadBuy == 1)

random_indices = np.random.choice(badBuy_indices, sample_size, replace=False)

Non_badBuy_sample = df.loc[random_indices]

print(Non_badBuy_sample['IsBadBuy'])

Badbuys = df.loc[df['IsBadBuy']==1]

sample_data = Badbuys.append(Non_badBuy_sample,ignore_index=True)
X = sample_data.drop(['IsBadBuy'],axis = 1)
y = sample_data['IsBadBuy']

X_mat = X.as_matrix()
X_train, X_test, y_train, y_test = train_test_split(X_mat, y, test_size=0.3, stratify=y, random_state=rs)



11017    False
30337    False
13004    False
3428     False
8843     False
33793    False
23531    False
33537    False
3922     False
40455    False
16035    False
9800     False
12952    False
37642    False
23440    False
15470    False
31305    False
6358     False
16718    False
37154    False
23510    False
18590    False
7589     False
12502    False
25413    False
17816    False
19407    False
32994    False
7142     False
8107     False
         ...  
36529    False
27592    False
19053    False
10934    False
2273     False
30536    False
17915    False
22998    False
20437    False
7857     False
925      False
16460    False
14223    False
24275    False
20914    False
3363     False
33158    False
11445    False
9890     False
34328    False
31221    False
9339     False
36500    False
11366    False
31110    False
11031    False
37174    False
28172    False
30547    False
15845    False
Name: IsBadBuy, Length: 5301, dtype: bool


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [22]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score

# simple decision tree training
model = DecisionTreeClassifier(random_state=rs)
model.fit(X_train, y_train)

print("Train accuracy:", model.score(X_train, y_train))


print("Test accuracy:", model.score(X_test, y_test))


Train accuracy: 1.0
Test accuracy: 0.56177302734989


In [23]:
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       False       0.56      0.56      0.56      1590
        True       0.56      0.56      0.56      1591

   micro avg       0.56      0.56      0.56      3181
   macro avg       0.56      0.56      0.56      3181
weighted avg       0.56      0.56      0.56      3181



In [24]:
#Showing Feature importance
import numpy as np

# grab feature importances from the model and feature name from the original X
importances = model.feature_importances_
feature_names = X.columns

# sort them out in descending order
indices = np.argsort(importances)
indices = np.flip(indices, axis=0)

# limit to 20 features
indices = indices[:20]

for i in indices:
    print(feature_names[i], ':', importances[i])

VehOdo : 0.10028847423940246
MMRCurrentRetailRatio : 0.08210880014858465
MMRCurrentAuctionAveragePrice : 0.07064678391857147
MMRAcquisitionAuctionAveragePrice : 0.06494301684484988
MMRAcquisitionRetailAveragePrice : 0.06465306008946631
MMRCurrentAuctionCleanPrice : 0.0631094450009982
VehYear : 0.060835007514111486
WarrantyCost : 0.060808088854541295
MMRCurrentRetailCleanPrice : 0.057879665690778945
MMRAcquisitionAuctionCleanPrice : 0.05756482163985702
MMRAcquisitonRetailCleanPrice : 0.055714194778263754
MMRCurrentRetailAveragePrice : 0.05203250402590111
IsOnlineSale : 0.010735481871975308
Transmission_AUTO : 0.002732078730954802
VehBCost_7500 : 0.002669264608471192
Transmission_MANUAL : 0.0022790719001501464
VehBCost_6800 : 0.0022257501236550865
VehBCost_4200 : 0.002037567552221229
VehBCost_5400 : 0.0020007815579037255
VehBCost_5700 : 0.0018099141346059125


In [25]:
import pydot
from io import StringIO
from sklearn.tree import export_graphviz

# visualize
dotfile = StringIO()
export_graphviz(model, out_file=dotfile, feature_names=X.columns)
graph = pydot.graph_from_dot_data(dotfile.getvalue())
graph[0].write_png("tree.png")

ModuleNotFoundError: No module named 'pydot'

In [26]:
#pre-pruning decision tree
model = DecisionTreeClassifier(max_depth=3, random_state=rs)
model.fit(X_train, y_train)

print("Train accuracy:", model.score(X_train, y_train))
print("Test accuracy:", model.score(X_test, y_test))

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

Train accuracy: 0.6152809594394286
Test accuracy: 0.6079849104055328
              precision    recall  f1-score   support

       False       0.60      0.68      0.63      1590
        True       0.62      0.54      0.58      1591

   micro avg       0.61      0.61      0.61      3181
   macro avg       0.61      0.61      0.61      3181
weighted avg       0.61      0.61      0.61      3181



In [27]:
# grab feature importance from the model and feature name from the original X
importances = model.feature_importances_
feature_names = X.columns

# sort them out in descending order
indices = np.argsort(importances)
indices = np.flip(indices, axis=0)

# limit to 20 features
indices = indices[:20]

for i in indices:
    print(feature_names[i], ':', importances[i])

# visualize
dotfile = StringIO()
export_graphviz(model, out_file=dotfile, feature_names=X.columns)
graph = pydot.graph_from_dot_data(dotfile.getvalue())
graph[0].write_png("week3_dt_viz.png") # saved in the following file

VehYear : 0.7102654015581602
MMRCurrentAuctionCleanPrice : 0.11443865338756819
VehOdo : 0.10349084174182029
MMRCurrentAuctionAveragePrice : 0.04596229232282906
MMRAcquisitionAuctionAveragePrice : 0.02584281098962225
VehBCost_3870 : 0.0
VehBCost_3890 : 0.0
VehBCost_3885 : 0.0
VehBCost_3880 : 0.0
VehBCost_38785 : 0.0
VehBCost_3875 : 0.0
VehBCost_3860 : 0.0
VehBCost_3865 : 0.0
VehBCost_3898 : 0.0
VehBCost_3855 : 0.0
VehBCost_3850 : 0.0
VehBCost_3845 : 0.0
VehBCost_3895 : 0.0
VehBCost_3910 : 0.0
VehBCost_3900 : 0.0


NameError: name 'StringIO' is not defined

In [28]:
test_score = []
train_score = []

# check the model performance for max depth from 2-20
for max_depth in range(2, 21):
    model = DecisionTreeClassifier(max_depth=max_depth, random_state=rs)
    model.fit(X_train, y_train)
    
    test_score.append(model.score(X_test, y_test))
    train_score.append(model.score(X_train, y_train))

In [29]:
import matplotlib.pyplot as plt

# plot max depth hyperparameter values vs training and test accuracy score
plt.plot(range(2, 21), train_score, 'b', range(2,21), test_score, 'r')
plt.xlabel('max_depth\nBlue = training acc. Red = test acc.')
plt.ylabel('accuracy')
plt.show()

<Figure size 640x480 with 1 Axes>

In [30]:
from sklearn.model_selection import GridSearchCV

In [31]:
# grid search CV
params = {'criterion': ['gini', 'entropy'],
          'max_depth': range(2, 7),
          'min_samples_leaf': range(20, 60, 10)}

cv = GridSearchCV(param_grid=params, estimator=DecisionTreeClassifier(random_state=rs), cv=10)
cv.fit(X_train, y_train)

print("Train accuracy:", cv.score(X_train, y_train))
print("Test accuracy:", cv.score(X_test, y_test))

# test the best model
y_pred = cv.predict(X_test)
print(classification_report(y_test, y_pred))

# print parameters of the best model
print(cv.best_params_)

Train accuracy: 0.644522301576607
Test accuracy: 0.6104998428167243
              precision    recall  f1-score   support

       False       0.61      0.61      0.61      1590
        True       0.61      0.61      0.61      1591

   micro avg       0.61      0.61      0.61      3181
   macro avg       0.61      0.61      0.61      3181
weighted avg       0.61      0.61      0.61      3181

{'criterion': 'gini', 'max_depth': 6, 'min_samples_leaf': 30}
